# Ensemble LGBM + CatBoost (feature 54)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "./input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/train.csv
/kaggle/input/test.csv


In [2]:
data_path = '/kaggle/input/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [3]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [4]:
drop_features = []

In [5]:
# "특정 시술 유형" 제거
# "주요 시술 유형" 생성

cat_features = [ 
    "시술 당시 나이", # FI top4
    "시술 유형",
    "주요 시술 유형",
    "배란 유도 유형",
    "난자 출처", # FI top
        
    "난자 기증자 나이", #FI top (알수 없음)

    "정자 출처",
    "정자 기증자 나이",
    "시술 시기 코드", 
    "배아 생성 주요 이유"
]

print(len(cat_features))

10


In [6]:
# "IVF 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수" 제거
# "DI 시술 횟수", "DI 임신 횟수", "DI 출산 횟수" 제거
# "IVF 임신 성공률", "IVF 출산 성공률", "IVF 실패 횟수", "IVF 유산 횟수", "DI 임신 성공률", "DI 출산 성공률", "DI 실패 횟수", "DI 유산 횟수" 생성 

ord_features = [
    "IVF 임신 성공률", 
    "IVF 출산 성공률", 
    "IVF 실패 횟수", 
    "IVF 유산 횟수", 
    "DI 임신 성공률", 
    "DI 출산 성공률", 
    "DI 실패 횟수", 
    "DI 유산 횟수",
    
    "클리닉 내 총 시술 횟수", #FI top
    "총 시술 횟수",
    "총 임신 횟수",
    "총 출산 횟수"
]

print(len(ord_features))

# drop_features = drop_features + ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수"]

12


In [7]:
num_features = [
    "이식된 배아 수", # FI top1
    "해동 난자 수", #FI top
    "수집된 신선 난자 수", #FI top7
    "파트너 정자와 혼합된 난자 수", #FI top
    "배아 이식 경과일", # FI top2,
    
    
    "저장된 배아 수", # FI top3
    "미세주입 후 저장된 배아 수", #FI top
    "총 생성 배아 수", # FI top5
    "미세주입된 난자 수", #FI top8
    "혼합된 난자 수", #FI top
    "미세주입에서 생성된 배아 수", #FI top
    "미세주입 배아 이식 수", #FI top
    "해동된 배아 수", #FI top
    "난자 채취 경과일", #FI top
    "임신 시도 또는 마지막 임신 경과 연수", #FI top

    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 해동 경과일",
    "저장된 신선 난자 수",
    "기증자 정자와 혼합된 난자 수"
]

print(len(num_features))

# drop_features = drop_features + ["난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수"]

20


In [8]:
# "남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
# "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증" 제거
# "부부 주 불임 원인", "부부 부 불임 원인" 제거
# "남성 불임 원인", "여성 불임 원인", "부부 불임 원인" 생성
# "PGD 시술 여부", "PGS 시술 여부" 제거 

bin_features = [
    '배란 자극 여부',
    '단일 배아 이식 여부', #FI top
    '착상 전 유전 진단 사용 여부',
    '남성 불임 원인',
    '여성 불임 원인',
    '부부 불임 원인',
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",

    "불명확 불임 원인",
    "기증 배아 사용 여부",
    '착상 전 유전 검사 사용 여부',
    "대리모 여부"
]

print(len(bin_features))

drop_features = drop_features + ["PGD 시술 여부", "PGS 시술 여부"]

12


In [9]:
# 시술 유형(cat feature) 피처 엔지니어링 

all_data["주요 시술 유형"] = all_data["특정 시술 유형"].apply(lambda x: 
    "ICSI" if "ICSI" in str(x) else 
    "IVF" if "IVF" in str(x) else 
    "IUI" if "IUI" in str(x) else 
    "Other"
)

drop_features = drop_features + ['특정 시술 유형']

all_data['주요 시술 유형'].value_counts(normalize=True, dropna=False) * 100

주요 시술 유형
ICSI     50.208130
IVF      36.909168
Other    10.511578
IUI       2.371124
Name: proportion, dtype: float64

In [10]:
# 과거 이력(ord feature) 피처 엔지니어링

past_features = ["IVF 시술 횟수", "DI 시술 횟수", "IVF 임신 횟수", "DI 임신 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수", "클리닉 내 총 시술 횟수"]:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)
        
for col in past_features:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)

all_data["IVF 임신 성공률"] = all_data["IVF 임신 횟수"] / (all_data["IVF 시술 횟수"] + 1)
all_data["IVF 출산 성공률"] = all_data["IVF 출산 횟수"] / (all_data["IVF 임신 횟수"] + 1)
all_data["IVF 실패 횟수"] = all_data["IVF 시술 횟수"] - all_data["IVF 임신 횟수"]
all_data["IVF 유산 횟수"] = all_data["IVF 임신 횟수"] - all_data["IVF 출산 횟수"]

all_data["DI 임신 성공률"] = all_data["DI 임신 횟수"] / (all_data["DI 시술 횟수"] + 1)
all_data["DI 출산 성공률"] = all_data["DI 출산 횟수"] / (all_data["DI 임신 횟수"] + 1)
all_data["DI 실패 횟수"] = all_data["DI 시술 횟수"] - all_data["DI 임신 횟수"]
all_data["DI 유산 횟수"] = all_data["DI 임신 횟수"] - all_data["DI 출산 횟수"]

drop_features = drop_features + past_features

In [11]:
# 불임 원인(bin features) 피처 엔지니어링 

infertility_features = ["남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
                        "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
                        "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
                        "부부 주 불임 원인", "부부 부 불임 원인"]

all_data["남성 불임 원인"] = (
    all_data["남성 주 불임 원인"] + 
    all_data["남성 부 불임 원인"] + 
    all_data["불임 원인 - 남성 요인"] + 
    all_data["불임 원인 - 정자 농도"] + 
    all_data["불임 원인 - 정자 면역학적 요인"] + 
    all_data["불임 원인 - 정자 운동성"] + 
    all_data["불임 원인 - 정자 형태"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["여성 불임 원인"] = (
    all_data["여성 주 불임 원인"] + 
    all_data["여성 부 불임 원인"] + 
    all_data["불임 원인 - 난관 질환"] + 
    all_data["불임 원인 - 배란 장애"] + 
    all_data["불임 원인 - 여성 요인"] + 
    all_data["불임 원인 - 자궁경부 문제"] + 
    all_data["불임 원인 - 자궁내막증"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["부부 불임 원인"] = (
    all_data["부부 주 불임 원인"] + 
    all_data["부부 부 불임 원인"]
).apply(lambda x: 1 if x > 0 else 0)

drop_features = drop_features + infertility_features

print(all_data['남성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['여성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['부부 불임 원인'].value_counts(normalize=True, dropna=False) * 100)

남성 불임 원인
0    60.57855
1    39.42145
Name: proportion, dtype: float64
여성 불임 원인
0    66.416583
1    33.583417
Name: proportion, dtype: float64
부부 불임 원인
0    95.81546
1     4.18454
Name: proportion, dtype: float64


In [12]:
extracted_data = all_data.drop(columns=drop_features, errors='ignore')

In [13]:
extracted_features = extracted_data.columns
extracted_features

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '배란 자극 여부',
       '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
       '불명확 불임 원인', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수',
       '총 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일',
       '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '주요 시술 유형',
       'IVF 임신 성공률', 'IVF 출산 성공률', 'IVF 실패 횟수', 'IVF 유산 횟수', 'DI 임신 성공률',
       'DI 출산 성공률', 'DI 실패 횟수', 'DI 유산 횟수', '남성 불임 원인', '여성 불임 원인',
       '부부 불임 원인'],
      dtype='object')

In [14]:
len(extracted_features)

54

## 2. 피처 엔지니어링: 원 핫 인코딩

In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<346418x57 sparse matrix of type '<class 'numpy.float64'>'
	with 3464180 stored elements in Compressed Sparse Row format>

In [16]:
remaining_features = list(set(extracted_features) - set(cat_features))

In [17]:
print(extracted_data[remaining_features].dtypes)

총 생성 배아 수                float64
임신 시도 또는 마지막 임신 경과 연수    float64
배아 해동 경과일                float64
착상 전 유전 진단 사용 여부         float64
신선 배아 사용 여부              float64
저장된 신선 난자 수              float64
DI 유산 횟수                 float64
IVF 실패 횟수                float64
해동된 배아 수                 float64
착상 전 유전 검사 사용 여부         float64
DI 임신 성공률                float64
혼합된 난자 수                 float64
총 임신 횟수                  float64
난자 해동 경과일                float64
난자 혼합 경과일                float64
저장된 배아 수                 float64
미세주입에서 생성된 배아 수          float64
파트너 정자와 혼합된 난자 수         float64
클리닉 내 총 시술 횟수            float64
IVF 임신 성공률               float64
DI 출산 성공률                float64
이식된 배아 수                 float64
해동 난자 수                  float64
미세주입 배아 이식 수             float64
IVF 출산 성공률               float64
총 시술 횟수                  float64
단일 배아 이식 여부              float64
대리모 여부                   float64
여성 불임 원인                   int64
남성 불임 원인                   int64
미세주입된 난자 수

In [18]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(extracted_data[remaining_features].fillna(0)),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [19]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 4. ROC-AUC 정의

In [20]:
from sklearn.metrics import roc_auc_score

# LGBM 커스텀 평가 함수
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

In [21]:
# XGBoost 커스텀 평가 함수
def xgb_roc_auc(y_pred, dtrain):
    y_true = dtrain.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred)

## 6. 층화 K 폴드 설정

In [22]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

## 6. 모델 훈련

### 6-1. LGBM

In [23]:
max_params_lgb = {
    'bagging_fraction': 0.5676682710015916,
    'feature_fraction': 0.9534471026963212,
    'lambda_l1': 0.04811015026095178,
    'lambda_l2': 1.9483311603902702,
    'min_child_samples': 20,
    'min_child_weight': 35.098326662286894,
    'num_leaves': 24,
    'objective': 'binary',
    'learning_rate': 0.01,
    'bagging_freq': 1,
    'force_row_wise': True,
    'random_state': 1991
}

In [24]:
import lightgbm as lgb

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds_lgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds_lgb = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params_lgb,     # 최적 하이퍼파라미터
                          train_set=dtrain,          # 훈련 데이터셋
                          num_boost_round=2500,      # 부스팅 반복 횟수
                          valid_sets=dvalid,         # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[lgb.early_stopping(stopping_rounds=100)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds_lgb += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds_lgb[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds_lgb[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59605, number of negative: 171110
[LightGBM] [Info] Total Bins 747
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054567
[LightGBM] [Info] Start training from score -1.054567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1274]	valid_0's binary_logloss: 0.487832	valid_0's roc_auc: 0.740191
폴드 1 roc-auc : 0.7401909756048809

######################################## 폴드 2 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59606, number of negative: 171110
[LightGBM] [Info] Total Bins 746
[LightGBM] [Info] Number of data points in the train set: 230716, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054

### 6-2. CatBoost

In [25]:
max_params_catboost = {
    'bagging_temperature': 0.5716165060534971,
    'border_count': 85,
    'colsample_bylevel': 0.8865557564347857,
    'depth': 7,
    'l2_leaf_reg': 8.858400514532834,
    'learning_rate': 0.02633263003504225,
    'min_data_in_leaf': 2.553144027856933,
    'od_wait': 28.58805968658813,
    'random_strength': 4.86507834123245,
    'subsample': 0.7823541020534175,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 2500,
    'random_seed': 1991,
    'verbose': 200
}

In [26]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# OOF 방식으로 모델 훈련, 검증, 예측
oof_val_preds_catboost = np.zeros(X.shape[0])
oof_test_preds_catboost = np.zeros(X_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)
    
    cat_model = CatBoostClassifier(**max_params_catboost)
    cat_model.fit(dtrain, 
                  eval_set=dvalid, 
                  early_stopping_rounds=200, 
                  verbose=500)
    
    # 테스트 데이터 예측
    oof_test_preds_catboost += cat_model.predict_proba(X_test)[:, 1] / folds.n_splits
    # 검증 데이터 예측
    oof_val_preds_catboost[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds_catboost[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
0:	test: 0.6778780	best: 0.6778780 (0)	total: 86.9ms	remaining: 3m 37s
500:	test: 0.7367453	best: 0.7367453 (500)	total: 18.7s	remaining: 1m 14s
1000:	test: 0.7394728	best: 0.7394728 (1000)	total: 37.4s	remaining: 56s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7396210087
bestIteration = 1050

Shrink model to first 1051 iterations.
폴드 1 ROC-AUC : 0.7396210086919794

######################################## 폴드 2 / 폴드 10 ########################################
0:	test: 0.6796715	best: 0.6796715 (0)	total: 37.2ms	remaining: 1m 33s
500:	test: 0.7372720	best: 0.7372720 (500)	total: 18s	remaining: 1m 11s
1000:	test: 0.7388582	best: 0.7389099 (975)	total: 36.7s	remaining: 54.9s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7389099072
bestIteration = 975

Shrink model to first 976 iterations.
폴드 2 ROC-AUC : 0.738909907181249

##########################

## 7. 최종 결과

In [27]:
print('LightGBM OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, oof_val_preds_lgb))

LightGBM OOF 검증 데이터 ROC-AUC : 0.7403453330156159


In [28]:
print('CatBoost OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, oof_val_preds_catboost))

CatBoost OOF 검증 데이터 ROC-AUC : 0.7399294382356356


In [29]:
correlation = np.corrcoef(oof_val_preds_lgb, oof_val_preds_catboost)[0,1]
print(f"LightGBM과 CatBoost 예측값의 상관계수: {correlation}")

LightGBM과 CatBoost 예측값의 상관계수: 0.9954121001049907


In [35]:
# LightGBM과 XGBoost 앙상블 예측 (가중 평균)
ensemble_preds = oof_val_preds_lgb * 0.7 + oof_val_preds_catboost * 0.3  # LGB 60%, XGB 40%

# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7404344882415504


In [31]:
# LightGBM과 XGBoost 앙상블 예측 (가중 평균)
ensemble_preds = oof_val_preds_lgb * 0.75 + oof_val_preds_catboost * 0.25  # LGB 60%, XGB 40%

# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7404324630603752


In [32]:
from scipy.optimize import minimize

# 최적의 가중치를 찾는 함수
def optimize_weights(weights):
    blended_preds = oof_val_preds_lgb * weights[0] + oof_val_preds_catboost * weights[1]
    return -roc_auc_score(y, blended_preds)  # 최대화 -> 최소화 문제로 변환

# 제약 조건 (가중치 합 1)
constraints = ({'type': 'eq', 'fun': lambda w: w[0] + w[1] - 1})

# 초기값 (LightGBM 60%, XGBoost 40%)
init_weights = [0.7, 0.3]

# 최적화 실행
best_weights = minimize(optimize_weights, init_weights, constraints=constraints, bounds=[(0, 1), (0, 1)])
print(f"최적 가중치: {best_weights.x}")

최적 가중치: [0.70000469 0.29999531]


In [33]:
# LightGBM과 XGBoost 앙상블 예측 (가중 평균)
ensemble_preds = oof_val_preds_lgb * 0.70000469 + oof_val_preds_catboost * 0.29999531  # LGB 60%, XGB 40%

# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7404344901476032


In [ ]:
ensemble_test_preds = oof_test_preds_lgb * 0.7 + oof_test_preds_catboost * 0.3
submission['probability'] = ensemble_test_preds
submission.to_csv('ensemble_lgbm_cat_submission_54.csv')